In [1]:
import QUANTAXIS as qa
from QIFIAccount import QIFI_Account,QA_Position
from QIFIAccount.QASimAccount import QIFI_SIM_Account
import pandas as pd

提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改


In [2]:
username='myacc21'
my=QIFI_SIM_Account(username=username,password=username)
my.initial()

Create new Account


In [3]:
df=pd.read_excel("E:\\leo\\投资\\2021\\股票持仓分析20210509-快照20210520.xlsx",sheet_name="持仓明细详情",header=6)
df=df[df['pos_num']>0]
code_cn_list=list(x.split(".")[0] for x in df.code[df['currency']=='CNY'].unique().tolist())
code_cn_list
code_hk_list=list(x.split(".")[0] for x in df.code[df['currency']=='HK'].unique().tolist())
code_hk_list=[i.rjust(5,'0') if len(i)==4  else i for i in code_hk_list]

In [4]:
def add_position(stock):
    code=stock['code'].split('.')
    if len(code[0])==6 and stock['currency']=='CNY':
        code=code[0]
        market_type=qa.MARKET_TYPE.STOCK_CN
        exchange_id=qa.EXCHANGE_ID.SZSE if stock['code'][0] in (0,3) else qa.EXCHANGE_ID.SSE
    elif len(code[0])==6:
        code=code[0]
        market_type='stock_cn_b'
        exchange_id=qa.EXCHANGE_ID.SZSE
    else:
        code=str(code[0]).rjust(5,'0')
        market_type='stock_hk'
        exchange_id='hk'
    pos=QA_Position(
                code=code,
                account_cookie='myacc6',
                market_type=market_type,
                exchange_id=exchange_id,
                volume_long_his=stock['pos_num'],
                volume_long_today=0,
                open_price_long=stock['cost'],
                name=stock['name'])
    my.add_position(pos)
def s(x):
    print(type(x['code']),x['code'].split('.'))
def add_position1(stock):
    code=stock['code'].split('.')
#     print(code[0],stock['currency'],stock['code'][0])
    if len(code[0])==6 and stock['currency']=='CNY':
        code=code[0]
        market_type=qa.MARKET_TYPE.STOCK_CN
        exchange_id=qa.EXCHANGE_ID.SZSE if code[0] in ('0','3') else qa.EXCHANGE_ID.SSE
    elif len(code[0])==6:
        code=code[0]
        market_type='stock_cn_b'
        exchange_id=qa.EXCHANGE_ID.SZSE
    else:
        code=str(code[0]).rjust(5,'0')
        market_type='stock_hk'
        exchange_id='hk'
    pos=QA_Position(
                code=code,
                account_cookie='myacc21',
                market_type=market_type,
                exchange_id=exchange_id,
                volume_long_his=stock['pos_num'],
                volume_long_today=0,
                open_price_long=stock['cost'],
                name=stock['name'])
#     print(pos.message)
    my.add_position(pos)

In [5]:
df.groupby('code').agg({'pos_num':'sum','cost':'mean','name':'first','currency':'first'}).reset_index().apply(lambda x:add_position1(x),axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
dtype: object

In [20]:
df=pd.DataFrame(my.position_msg).T
df1=df[['code','name','exchange_id','volume_long_his','pos_long_his','open_price_long','last_price','position_profit','position_cost_long']].assign(balance=df['last_price']*df['pos_long_his'])
df1

,code,name,exchange_id,volume_long_his,pos_long_his,open_price_long,last_price,position_profit,position_cost_long,balance
stock_cn.000333,000333,美的集团,szse,374,374,51.69,73.17,8033.52,19332.1,27365.6
stock_cn.000961,000961,中南建设,szse,13500,13500,7.256,6,-16956,97956,81000
stock_cn.002027,002027,分众传媒,szse,12700,12700,5.986,9.15,40182.8,76022.2,116205
stock_cn.002353,002353,杰瑞股份,szse,300,300,32.265,45.26,3898.5,9679.5,13578
stock_hk.00700,00700,腾讯控股,hk,400,400,474.321,493.403,7633.16,189728,197361
stock_hk.01088,01088,中国神华,hk,12000,12000,14.96,14.7403,-2636.73,179520,176883
stock_hk.01516,01516,融创服务,hk,32,32,17.16,22.3,164.481,549.12,713.601
stock_cn.159920,159920,恒生ETF,sse,44700,44700,1.525,1.523,-89.4,68167.5,68078.1
stock_cn.160322,160322,港股精选,sse,2800,2800,1.796,1.714,-229.6,5028.8,4799.2
stock_cn.160631,160631,银行指基,sse,2447,2447,1,1.046,112.562,2447,2559.56


In [13]:
my.settle()

settle


In [18]:
my.balance

3088730.161620059

In [19]:
my.position_profit

353614.4486200593

In [17]:
day='2021-06-25'
for code in code_cn_list:
    price=qa.QA_fetch_get_stock_day('tdx',code,day,day)
    price.apply(lambda x:my.on_price_change(code,x.close,datetime=x.date),axis=1)
for code in code_hk_list:
    price=qa.QA_fetch_get_hkstock_day('tdx',code,day,day) if len(code)==5 else qa.QA_fetch_get_stock_day('tdx',code,day,day)
    price.apply(lambda x:my.on_price_change(code,x.close*0.8244,datetime=x.date),axis=1)

In [23]:
########################得到汇率####################
import http.client
conn = http.client.HTTPSConnection("hq.sinajs.cn")
payload = ''
headers = {}
conn.request("GET", "/rn=1624697495013list=fx_shkdcny", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("gbk"))

var hq_str_fx_shkdcny="11:39:21,0.831100,0.832100,0.831200,0,0.831100,0.831100,0.831100,0.831100,港元兑人民币即期汇率,-0.010000,-0.000100,0,OTC Data Services Editorial Team Calculated Cross Rates. New York,0.818600,0.818600,********,2021-06-26";



In [24]:
data

b'var hq_str_fx_shkdcny="11:39:21,0.831100,0.832100,0.831200,0,0.831100,0.831100,0.831100,0.831100,\xb8\xdb\xd4\xaa\xb6\xd2\xc8\xcb\xc3\xf1\xb1\xd2\xbc\xb4\xc6\xda\xbb\xe3\xc2\xca,-0.010000,-0.000100,0,OTC Data Services Editorial Team Calculated Cross Rates. New York,0.818600,0.818600,********,2021-06-26";\n'

In [25]:
k=data.decode("gbk")

In [35]:
k[k.find("\"")+1:k.rfind("\"")].split(",")

['11:39:21',
 '0.831100',
 '0.832100',
 '0.831200',
 '0',
 '0.831100',
 '0.831100',
 '0.831100',
 '0.831100',
 '港元兑人民币即期汇率',
 '-0.010000',
 '-0.000100',
 '0',
 'OTC Data Services Editorial Team Calculated Cross Rates. New York',
 '0.818600',
 '0.818600',
 '********',
 '2021-06-26']

In [31]:
k.rfind("\"")

233